# Lab 6: Building A RAG Agent

## Setup Components

In [1]:
## Load Environment Table
from dotenv import load_dotenv
load_dotenv(override=True)  # take environment variables

True

In [2]:
# Load large language model 
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4.1-nano", model_provider="openai")

In [ ]:
## 
from langchain_ibm import ChatWatsonx
import os

llm = ChatWatsonx(
    model_id="ibm/granite-34b-code-instruct", 
    url="https://us-south.ml.cloud.ibm.com",
    project_id=os.environ['WATSONX_PROJECT_ID']
)

/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-34b-code-instruct' is in deprecated state from 2025-04-16 until 2025-07-17. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [ ]:
%pip install langchain_huggingface

In [3]:
# Selecting Embedding model
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Users/hizkiafebianto/Documents/GitHub/revou-gen-ai-tutorial/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Initiate Vector Code
# In-memory 

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embedding_model)

## Document Loaders

In [2]:
%pip install -qU pypdf


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
##
file_path = "./docs/Tentang Dexa Medica.pdf"

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [6]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'creationdate': '2025-05-25T09:21:53+00:00', 'title': 'Dexa Medica • Dexa Group', 'moddate': '2025-05-25T09:21:53+00:00', 'source': './docs/Tentang Dexa Medica.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}

Gambaran Umum
Dexa Medica, salah satu perusahaan farmasi terbesar di Indonesia, telah secara konsisten
menggunakan kemampuan penelitiannya yang canggih untuk memprioritaskan kebutuhan
kesehatan masyarakat Indonesia. Mottonya — pemanfaatan keahlian untuk mempromosikan
kesehatan — adalah konsekuensi alami dari komitmen jangka panjangnya meningkatkan
perawatan kesehatan bagi semua orang. Perusahaan dan tim manajemen Dexa Medica dipandu
oleh empat prinsip inti yaitu kejujuran, kepercayaan, dedikasi, dan komitmen.
Dexa Medica didirikan di Palembang (Sumatera Selatan, Indonesia) pada tahun 1969 oleh
almarhum Letkol apt. D

In [7]:
docs = loader.load()
docs

[Document(metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'creationdate': '2025-05-25T09:21:53+00:00', 'title': 'Dexa Medica • Dexa Group', 'moddate': '2025-05-25T09:21:53+00:00', 'source': './docs/Tentang Dexa Medica.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='Gambaran Umum\nDexa Medica, salah satu perusahaan farmasi terbesar di Indonesia, telah secara konsisten\nmenggunakan kemampuan penelitiannya yang canggih untuk memprioritaskan kebutuhan\nkesehatan masyarakat Indonesia. Mottonya — pemanfaatan keahlian untuk mempromosikan\nkesehatan — adalah konsekuensi alami dari komitmen jangka panjangnya meningkatkan\nperawatan kesehatan bagi semua orang. Perusahaan dan tim manajemen Dexa Medica dipandu\noleh empat prinsip inti yaitu kejujuran, kepercayaan, dedikasi, dan komitmen.\nDexa Medica didirikan di Palembang (Sumatera Selatan, Indonesia) pad

## Splitting

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

18

In [9]:
for split in all_splits:
    print(split)

page_content='Gambaran Umum
Dexa Medica, salah satu perusahaan farmasi terbesar di Indonesia, telah secara konsisten
menggunakan kemampuan penelitiannya yang canggih untuk memprioritaskan kebutuhan
kesehatan masyarakat Indonesia. Mottonya — pemanfaatan keahlian untuk mempromosikan
kesehatan — adalah konsekuensi alami dari komitmen jangka panjangnya meningkatkan
perawatan kesehatan bagi semua orang. Perusahaan dan tim manajemen Dexa Medica dipandu' metadata={'producer': 'Skia/PDF m136', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'creationdate': '2025-05-25T09:21:53+00:00', 'title': 'Dexa Medica • Dexa Group', 'moddate': '2025-05-25T09:21:53+00:00', 'source': './docs/Tentang Dexa Medica.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1', 'start_index': 0}
page_content='perawatan kesehatan bagi semua orang. Perusahaan dan tim manajemen Dexa Medica dipandu
oleh empat prinsip inti yaitu kejujuran, kep

### Strategy for Splitting Documents
1. a 
2. b

## Embedding and Indexing

In [10]:
vector_store.add_documents(all_splits)

['9eb5b0d0-dfbb-4ea2-9b37-8653f928303c',
 'c9839ecb-f752-4f6f-aecc-b27cd0168c7d',
 '666eacc5-619b-406d-ada3-3857b194cfcd',
 'f7cf4543-df39-4e60-a21d-517727c61aae',
 '109d825e-4231-496e-bec7-1e45ac38c21c',
 'f0fd3ff4-8512-4714-9faa-2c2894d57589',
 '1f883262-f44f-4620-8023-ccfb9ae5300b',
 '7486da72-d3f7-447b-ae89-c700d65f255c',
 'd5b8254f-62ec-4100-adce-b20b6b12b695',
 '54e2e6a8-a837-4efc-bfa7-6db5f965a22d',
 'fec4c125-1aa6-438e-b1f5-c90abb9b803a',
 '8f5eeb21-be51-4c56-8141-b28d16917b7e',
 '2a74211e-300b-4c0f-a9a9-7298eef01dfd',
 'c1bbb056-238e-4ae1-b37f-382c2feb0e2d',
 '2999604a-5e55-42ce-baa9-0349ebae9cce',
 '5a98ed47-65c9-447c-bc52-01f7c9932af8',
 '31766914-1b54-4079-bd4c-e6cc223283bb',
 'f730d46e-bb15-49b5-9541-62a43bac0840']

In [11]:

# Step 8: Accept user input and perform semantic search
def query_rag_system(query: str):
    docs = vector_store.similarity_search(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = f"Answer the question based on the context below:\n\n{context}\n\nQuestion: {query}"
    return llm.invoke(prompt)

user_query = "Siapa pendiri Dexa?"
response = query_rag_system(user_query)
print("\nAnswer:\n", response)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Answer:
 content='Pendiri Dexa Medica adalah almarhum Letkol apt. Drs. Rudy Soetikno.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 536, 'total_tokens': 558, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370', 'id': 'chatcmpl-Bc6hulhbDJo3TxniGlaHpG7EwMMmO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--89622266-d0ab-4079-b320-a744274fa1c3-0' usage_metadata={'input_tokens': 536, 'output_tokens': 22, 'total_tokens': 558, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
